# Prompt Engineering : Advanced Prompting avec OpenAI

Dans ce notebook, nous allons tester différentes techniques avancées de **prompt engineering**:
- **Zero-shot prompting**
- **Few-shot prompting**
- **Chain-of-thought** (CoT)
- **Self-refine** (ou auto-amélioration)

Nous utiliserons la **nouvelle API** de la bibliothèque `openai` (>=1.0.0) via la classe `OpenAI` et ses méthodes de chat (`client.chat.completions.create`).


# 2. Prompt Engineering : Techniques Avancées

**Navigation** : [<< Precedent](1_OpenAI_Intro.ipynb) | [Index](../../README.md) | [Suivant >>](3_Structured_Outputs.ipynb)

## Objectifs d'apprentissage

A la fin de ce notebook, vous saurez :
1. Maîtriser les techniques de zero-shot et few-shot prompting
2. Comprendre et appliquer le chain-of-thought (CoT)
3. Implémenter le self-refine pour l'amélioration itérative
4. Distinguer modèles chat et modèles de raisonnement

### Prerequis
- Notebook 1 (Introduction a l'IA generative)
- Python 3.10+
- Cle API OpenAI configuree

### Duree estimee : 60 minutes

---

# Prompt Engineering : Advanced Prompting avec OpenAI

## Installation des dépendances

Avant de commencer, nous devons installer les bibliothèques Python nécessaires.

**Packages requis** :
- **openai** : Bibliothèque officielle pour interagir avec l'API OpenAI (>=1.0.0)
- **tiktoken** : Encodeur de tokens pour compter et gérer les tokens GPT
- **python-dotenv** : Gestion sécurisée des clés API via fichiers .env

> **Note de sécurité** : Ne jamais inclure vos clés API directement dans le code. Toujours utiliser un fichier  exclu du contrôle de version (.gitignore).

In [1]:
# ============================
# Cellule 1 : Installation
# ============================

%pip install openai tiktoken python-dotenv
# Remarque : Aucune fin de ligne en commentaire pour éviter l'erreur


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


## Pourquoi le Prompt Engineering ?

Le **prompt engineering** est l'art de formuler des instructions efficaces pour obtenir les meilleures reponses des modeles de langage. C'est une competence essentielle car :

1. **Impact direct sur la qualite** : Un bon prompt peut transformer une reponse mediocre en resultat excellent
2. **Economie de tokens** : Des prompts bien concus reduisent les iterations et donc les couts
3. **Reproductibilite** : Des techniques structurees permettent des resultats coherents

### Progression de ce notebook

| Technique | Complexite | Cas d'usage |
|-----------|------------|-------------|
| Zero-shot | Simple | Questions generales, taches courantes |
| Few-shot | Moyenne | Format specifique, style personnalise |
| Chain-of-thought | Moyenne | Raisonnement, mathematiques, logique |
| Self-refine | Avancee | Code, textes critiques, haute qualite |

> **Documentation officielle** : [OpenAI Prompt Engineering Guide](https://platform.openai.com/docs/guides/prompt-engineering)

In [2]:
# ============================
# Cellule 2 : Configuration
# ============================

import os
import sys
from dotenv import load_dotenv

# Charger la configuration depuis le fichier .env (dans le répertoire parent GenAI/)
load_dotenv('../.env')

# On suppose que ton .env contient :
# OPENAI_API_KEY=sk-xxxxxx
# (ou autre variable si tu utilises Azure)
#
# Récupère la clé d'API
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("Clé API introuvable. Vérifie ton fichier .env.")

# Mode batch pour exécution non-interactive (Papermill, tests automatisés)
# Détection automatique si exécution via Papermill ou si stdin non disponible
def is_interactive():
    """Détecte si l'exécution est interactive (terminal) ou batch (Papermill)"""
    try:
        # Check if running in Papermill
        import __main__
        if hasattr(__main__, '__file__') and 'papermill' in str(getattr(__main__, '__file__', '')).lower():
            return False
        # Check if stdin is available
        if not sys.stdin.isatty():
            return False
        return True
    except:
        return False

BATCH_MODE = os.getenv("BATCH_MODE", "false").lower() == "true" or not is_interactive()
print(f"Mode: {'BATCH' if BATCH_MODE else 'INTERACTIF'}")

Mode: BATCH


## Configuration de l'environnement

Avant de commencer les expérimentations, nous devons configurer l'accès à l'API OpenAI et gérer les modes d'exécution.

### Mode batch vs mode interactif

Le notebook supporte deux modes d'exécution :

- **Mode interactif** : Pour l'apprentissage, avec saisie utilisateur et expérimentation libre
- **Mode batch** : Pour l'exécution automatisée (Papermill, tests, CI/CD), sans interaction

La détection est automatique, mais vous pouvez forcer le mode batch via la variable d'environnement `BATCH_MODE=true` dans le fichier `.env`.

### Initialisation du client OpenAI

Le client OpenAI moderne (>=1.0.0) utilise une API orientée objet :

```python
client = OpenAI(api_key="...")
response = client.chat.completions.create(...)
```

**Paramètres importants** :
- `model` : Le modèle à utiliser (gpt-5-mini, o4-mini, etc.)
- `max_tokens` : Longueur maximale de la réponse (évite les réponses trop longues et coûteuses)
- `temperature` : Contrôle la créativité (0.0 = déterministe, 2.0 = très créatif)

Par défaut, nous utilisons `gpt-5-mini` configuré dans le fichier `.env`.

## Partie 1 : Zero-shot Prompting

Le **zero-shot prompting** est la technique la plus directe : on pose une question sans fournir d'exemples préalables.

### Quand utiliser Zero-shot ?

- Questions générales ou conversationnelles
- Tâches courantes bien comprises par le modèle (résumés, traductions simples)
- Prototypage rapide
- Budget tokens limité

Testons avec une demande simple : générer des idées de recettes végétariennes.

In [ ]:
# ============================
# Cellule 3 : Client OpenAI
# ============================

import openai
from openai import OpenAI

# Charger le modèle depuis .env ou utiliser gpt-5-mini par défaut
DEFAULT_MODEL = os.getenv("OPENAI_MODEL", "gpt-5-mini")
MODEL_NAME = DEFAULT_MODEL

# Instanciation du client
client = OpenAI(
    api_key=api_key,
)

print("Client OpenAI initialisé avec succès !")
print(f"Modèle par défaut: {MODEL_NAME}")

### Rappel des différences entre Zero-shot, Few-shot, Chain-of-thought et Self-refine

1. **Zero-shot Prompting**  
   - Aucune instruction ou exemple préalable (à part la demande de l'utilisateur).  
   - Simple et direct, mais parfois moins précis ou cohérent.

2. **Few-shot Prompting**  
   - Fournir quelques exemples “input → output” pour guider la réponse.  
   - Permet de **spécifier le format**, le style, ou le contenu souhaité.  
   - Améliore significativement la qualité des réponses sur des tâches complexes.

3. **Chain-of-thought (CoT)**  
   - On **incite** le modèle à détailler son raisonnement étape par étape.  
   - Souvent utile pour des questions de logique, mathématiques, programmation ou raisonnement complexe.  
   - Peut **augmenter** la cohérence et la justesse de la réponse (mais attention à ne pas divulguer ces “étapes” si elles sont confidentielles).

4. **Self-refine**  
   - Demander au modèle de s’auto-critiquer puis de proposer une réponse améliorée.  
   - Mise en œuvre en plusieurs appels (réponse initiale, re-demande d’analyse, ré-énoncé final).  
   - Intéressant pour du code, des textes longs, ou des situations nécessitant un contrôle qualité.

---


## Partie 2 : Few-shot Prompting

Le **few-shot prompting** consiste à fournir 2-3 exemples de la tâche souhaitée avant la vraie question.

### Mécanisme d'apprentissage en contexte

Le modèle :
1. Analyse les exemples fournis
2. Détecte le **pattern** (format, style, structure)
3. Applique ce pattern à la nouvelle question

C'est ce qu'on appelle **l'apprentissage en contexte** (in-context learning) : le modèle s'adapte sans modifier ses poids.

### Premier exemple : Rédaction d'emails professionnels

Nous allons guider le modèle à produire un email avec un format et un ton spécifiques.

In [4]:
# ============================
# Cellule 5 : Zero-shot
# ============================

prompt_1 = "Donne-moi 3 idées de recettes végétariennes à base de tomates."
response_1 = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "user", "content": prompt_1}
    ],
    # Contrôle du style
    max_tokens=400,
    temperature=0.7  # plus la température est haute, plus c'est créatif
)

print("=== Zero-shot Prompt ===")
print(f"Prompt: {prompt_1}\n")
print("Réponse du modèle :\n")
print(response_1.choices[0].message.content)


=== Zero-shot Prompt ===
Prompt: Donne-moi 3 idées de recettes végétariennes à base de tomates.

Réponse du modèle :

Voici trois idées de recettes végétariennes à base de tomates :

1. **Tarte à la tomate et à la moutarde** :
   - **Ingrédients** : Pâte feuilletée, tomates mûres, moutarde à l'ancienne, fromage râpé (comme le gruyère ou le comté), herbes de Provence, sel, poivre.
   - **Préparation** : Préchauffez le four à 200°C. Étalez la pâte feuilletée dans un moule à tarte. Badigeonnez le fond de tarte avec de la moutarde. Disposez les tranches de tomates uniformément sur la pâte. Saupoudrez de fromage râpé, d'herbes de Provence, de sel et de poivre. Enfournez pendant 25 à 30 minutes jusqu'à ce que la tarte soit dorée.

2. **Salade de tomates, mozzarella, et basilic (Caprese)** :
   - **Ingrédients** : Tomates mûres, mozzarella de bufflonne, feuilles de basilic frais, huile d'olive extra-vierge, vinaigre balsamique, sel, poivre.
   - **Préparation** : Coupez les tomates et la mozz

### Résultat du Zero-shot prompting

Le zero-shot prompting est la technique la plus simple : **aucun exemple préalable**, juste une instruction directe.

**Avantages** :
- Rapide à mettre en œuvre
- Fonctionne bien pour des tâches courantes (résumés, traductions, questions générales)
- Économique en tokens

**Limites** :
- Moins précis sur des tâches complexes ou spécialisées
- Le format de sortie peut être imprévisible
- Nécessite des prompts très clairs et bien formulés

Dans cet exemple, le modèle génère 3 recettes végétariennes à base de tomates sans aucun exemple préalable. La qualité dépend fortement de la clarté du prompt et de la capacité du modèle à comprendre le domaine.

## Partie 3 : Chain-of-thought (CoT)

Le **Chain-of-thought** demande explicitement au modèle de détailler son raisonnement étape par étape.

### Pourquoi le CoT fonctionne ?

Les modèles de langage sont entraînés sur des textes où les raisonnements sont explicités. En demandant les étapes intermédiaires, on active ce pattern et on améliore la précision.

### Applications du CoT

| Domaine | Exemple |
|---------|---------|
| Mathématiques | Résolution d'équations, problèmes de mots |
| Logique | Syllogismes, déductions |
| Programmation | Debugging, conception d'algorithmes |
| Analyse | Cas juridiques, diagnostics médicaux |

### Exemple : Problème arithmétique simple

Testons avec un calcul impliquant plusieurs étapes.

## Partie 4 : Self-refine (Auto-amélioration)

Le **Self-refine** exploite la capacité du modèle à critiquer et améliorer ses propres productions.

### Processus en deux étapes

1. **Génération initiale** : Produire une première version (potentiellement bugguée ou imparfaite)
2. **Critique et correction** : Analyser la première version, identifier les problèmes, proposer une amélioration

### Avantages du Self-refine

- **Qualité supérieure** : Deux passes donnent généralement de meilleurs résultats
- **Détection de bugs** : Le modèle peut repérer ses propres erreurs
- **Amélioration itérative** : Peut être répété plusieurs fois si nécessaire

### Compromis

- **Coût** : Double les tokens consommés (deux appels API)
- **Temps** : Latence multipliée
- **À utiliser pour** : Code critique, documents importants, tâches complexes

### Exemple : Code Python avec bug volontaire

In [5]:
# ============================
# Cellule X (NOUVELLE) : Few-shot supplémentaire
# ============================

few_shot_prompt_2 = """
Tu es un assistant spécialisé en rédaction d'e-mails professionnels.
Voici quelques exemples de style :

Exemple 1:
Q: Rédige un e-mail pour informer un client d'un retard de livraison
A: 
Sujet: Information concernant le retard de votre livraison

Bonjour [Nom du Client],

Nous tenions à vous informer que votre commande #1234 a pris du retard...
[...suite du mail...]

Exemple 2:
Q: Envoie un e-mail de remerciement pour un entretien d'embauche
A:
Sujet: Remerciements suite à notre entretien

Bonjour [Nom du Contact],

Je tiens à vous remercier pour le temps que vous m'avez accordé...
[...suite du mail...]

Maintenant, voici ma demande:

Q: Écris un e-mail pour informer un collaborateur d'un changement de planning et l'inviter à une réunion de suivi.
A:
"""

response_few_shot_2 = client.chat.completions.create(
    model=MODEL_NAME,  # ex. "gpt-4o-mini"
    messages=[
        {"role": "user", "content": few_shot_prompt_2}
    ],
    max_tokens=300,
    temperature=0.6
)

print("=== Exemple Few-shot (e-mail professionnel) ===")
print(response_few_shot_2.choices[0].message.content)


=== Exemple Few-shot (e-mail professionnel) ===
Sujet: Changement de planning et invitation à une réunion de suivi

Bonjour [Nom du Collaborateur],

Je souhaite vous informer d'un changement récent dans notre planning initial. Afin de discuter des ajustements nécessaires et de nous assurer que tout le monde est aligné, je vous invite à participer à une réunion de suivi.

La réunion aura lieu le [Date] à [Heure], dans [Lieu/Plateforme pour une réunion en ligne]. Nous aborderons les points suivants : [énumérer les sujets principaux]. Votre présence et vos contributions seront précieuses pour garantir le succès de ce projet.

Merci de bien vouloir confirmer votre disponibilité à cette date. N'hésitez pas à me contacter si vous avez des questions ou des préoccupations avant la réunion.

Cordialement,

[Votre Nom]  
[Votre Position]  
[Votre Entreprise]  
[Vos Coordonnées]


### Résultat du Few-shot : Format cohérent

Le modèle a reproduit fidèlement la structure des exemples :

1. **Sujet** : Clair et professionnel
2. **Salutation** : Formule de politesse appropriée
3. **Corps** : Structure logique (contexte → action → conclusion)
4. **Signature** : Complète avec coordonnées

**Observation clé** : Sans les exemples, le modèle aurait pu générer un email plus informel ou moins structuré. Le few-shot garantit la cohérence du format.

### Analyse du Few-shot prompting

Le few-shot prompting apporte une **amélioration significative** par rapport au zero-shot :

**Mécanisme** :
1. On fournit 2-3 exemples de la tâche souhaitée (paires question/réponse)
2. Le modèle apprend le **pattern** et le **format** attendu
3. Il applique ensuite ce pattern à la nouvelle question

**Avantages observables** :
- **Format cohérent** : Le modèle reproduit la structure des exemples (sujet, salutation, corps, signature)
- **Ton approprié** : Le style professionnel est maintenu
- **Contenu pertinent** : La réponse suit les conventions des exemples fournis

**Quand utiliser Few-shot ?**
- Tâches avec un format spécifique (emails, rapports, analyses structurées)
- Cas où le zero-shot donne des résultats trop variables
- Besoin de cohérence stylistique

**Compromis** : Chaque exemple consomme des tokens supplémentaires, donc à utiliser avec modération pour des contextes très longs.

## Partie 5 : Interactions conversationnelles

Au-delà des techniques de prompting, la structure des conversations avec le modèle est cruciale pour des applications interactives.

### Prompt simple sans mémoire

La première cellule interactive montre un échange **stateless** (sans état) : chaque prompt est indépendant, le modèle n'a aucune mémoire des interactions précédentes.

**Cas d'usage** :
- Tests rapides d'un modèle
- Questions indépendantes
- Prototypage de prompts

### Conversation avec mémoire

La cellule suivante introduit un mécanisme de **mémoire de conversation** essentiel pour les chatbots et assistants.

**Principe** :
1. Accumuler les messages dans une liste `current_messages`
2. À chaque tour, envoyer **tout l'historique** + le nouveau message
3. Ajouter la réponse du modèle à l'historique

**Structure des messages** :
```python
[
    {"role": "user", "content": "Je m'appelle Alice"},
    {"role": "assistant", "content": "Enchanté Alice !"},
    {"role": "user", "content": "Comment je m'appelle ?"},
    # Le modèle peut répondre "Vous vous appelez Alice" grâce à l'historique
]
```

**Attention** : L'historique consomme des tokens à chaque appel. Pour de longues conversations, il faut :
- Tronquer les messages anciens
- Résumer l'historique périodiquement
- Utiliser des techniques de compression (embeddings, résumés automatiques)

## Partie 6 : Modèles de raisonnement (2025)

Les **modèles de raisonnement** (reasoning models) comme `o4-mini` et `gpt-5-thinking` représentent une évolution majeure des LLMs.

### Différence fondamentale

Les modèles chat génèrent token par token en temps réel. Les modèles de raisonnement :
1. **Réfléchissent** en interne avant de répondre (pensées non visibles)
2. **Explorent** plusieurs pistes de réflexion
3. **Valident** leur raisonnement avant de produire la réponse finale

### Paramètre clé : reasoning_effort

Remplace le paramètre `temperature` pour les modèles de raisonnement :

| Valeur | Durée | Cas d'usage |
|--------|-------|-------------|
| `low` | Rapide | Questions simples, prototypage |
| `medium` | Modéré | Problèmes standards |
| `high` | Lent | Problèmes complexes, mathématiques avancées |

### Role "developer" obligatoire

Les modèles de raisonnement n'utilisent plus le role `system`. Le role `developer` configure le comportement global, et peut activer/désactiver le formatage de la pensée interne.

### Comparaison pratique

Testons le même problème avec un modèle chat (gpt-4o-mini) et un modèle de raisonnement (o4-mini).

In [ ]:
# ============================
# Cellule 7 : Chain-of-thought
# ============================

cot_prompt = """
Alice a 5 pommes, elle en jette 2, puis elle en donne 1 à Bob.
Bob lui rend ensuite 1 pomme.
Combien de pommes Alice a-t-elle à la fin ?
Donne directement la réponse sans étape intermédiaire.
"""

# Explique ton raisonnement étape par étape, puis donne la réponse finale.
response_3 = client.chat.completions.create(
    model="gpt-5-mini",
    messages=[
        {"role": "user", "content": cot_prompt}
    ],
    max_tokens=200,
    temperature=0.2  # on réduit la température pour moins de fantaisie
)

print("=== Chain-of-thought Prompt ===")
print("Réponse du modèle (avec raisonnement) :\n")
print(response_3.choices[0].message.content)

### Analyse du résultat CoT

**Observation** : Le prompt demandait une réponse directe SANS étapes intermédiaires, et le modèle a bien obéi en donnant simplement "3 pommes".

**Expérimentation recommandée** : Modifiez le prompt pour demander explicitement le raisonnement :

```python
cot_prompt = """
Alice a 5 pommes, elle en jette 2, puis elle en donne 1 à Bob.
Bob lui rend ensuite 1 pomme.
Combien de pommes Alice a-t-elle à la fin ?
Explique ton raisonnement étape par étape avant de donner la réponse finale.
"""
```

Vous devriez alors obtenir :
- Étape 1 : 5 - 2 = 3
- Étape 2 : 3 - 1 = 2
- Étape 3 : 2 + 1 = 3
- **Réponse : 3 pommes**

### Bonnes pratiques CoT

1. **Temperature basse** (0.0-0.3) pour les calculs et la logique
2. **Prompt explicite** : "Explique ton raisonnement étape par étape"
3. **Vérification** : Le raisonnement explicite permet de détecter les erreurs
4. **Masquage optionnel** : Pour les applications de production, on peut demander au modèle de ne pas révéler son raisonnement à l'utilisateur final

### Interprétation du Chain-of-thought

Le Chain-of-thought (CoT) est particulièrement efficace pour les **problèmes de raisonnement** :

**Analyse du résultat** :
Le modèle devrait avoir détaillé :
1. État initial : Alice a 5 pommes
2. Étape 1 : Elle en jette 2 → 5 - 2 = 3 pommes
3. Étape 2 : Elle en donne 1 à Bob → 3 - 1 = 2 pommes
4. Étape 3 : Bob lui rend 1 pomme → 2 + 1 = 3 pommes
5. **Réponse finale : 3 pommes**

**Pourquoi c'est important ?**
- **Transparence** : On peut vérifier le raisonnement étape par étape
- **Détection d'erreurs** : Si le résultat est faux, on peut identifier où le modèle s'est trompé
- **Confiance** : Le raisonnement explicite augmente la crédibilité
- **Debugging** : Facilite la correction du prompt si nécessaire

**Applications** :
- Calculs mathématiques
- Raisonnement logique
- Résolution de problèmes complexes
- Analyse de cas juridiques ou médicaux

**Note** : Temperature=0.2 garantit un raisonnement cohérent et reproductible.

In [7]:
# ============================
# Cellule 8a : Self-refine Step 1
# ============================

prompt_sr1 = """
Ecris une courte fonction Python pour calculer la somme d'une liste. 
Ajoute un bug volontaire dans le code. 
"""

response_sr1 = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[{"role": "user", "content": prompt_sr1}],
    max_tokens=300
)

buggy_code = response_sr1.choices[0].message.content

print("=== Self-refine (1) : Code buggy ===\n")
print(buggy_code)


=== Self-refine (1) : Code buggy ===

Voici une fonction Python pour calculer la somme des éléments d'une liste, avec un bug volontaire inclus :

```python
def somme_liste(liste):
    total = 0
    for i in range(len(liste)):
        total += liste[i]
    # Bug volontaire : ici, on décrémente le total au lieu de l'incrémenter
    total -= liste[0]
    return total

# Exemple d'utilisation
ma_liste = [1, 2, 3, 4]
print(somme_liste(ma_liste))  # La sortie sera incorrecte à cause du bug
```

Le bug volontaire ici est la décrémentation de `total` par le premier élément de la liste après avoir accumulé la somme de tous les éléments, ce qui conduit à un résultat incorrect.


#### 8b. Self-critique et amélioration

In [8]:
# ============================
# Cellule 8b : Self-refine Step 2
# ============================

prompt_sr2 = f"""
Voici un code Python qui contient un bug:

{buggy_code}

Peux-tu l'analyser, détecter le bug, proposer un correctif et une version améliorée du code ? 
Explique la correction.
"""

response_sr2 = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[{"role": "user", "content": prompt_sr2}],
    max_tokens=400,
    temperature=0.3
)

print("=== Self-refine (2) : Correction ===\n")
print(response_sr2.choices[0].message.content)


=== Self-refine (2) : Correction ===

Le code donné contient un bug dans la fonction `somme_liste`. Le problème se situe à la ligne où le total est décrémenté par le premier élément de la liste après avoir calculé la somme de tous les éléments. Cela entraîne un résultat incorrect, car la somme totale est réduite de manière inappropriée.

### Analyse du bug

Dans la fonction `somme_liste`, après avoir correctement additionné tous les éléments de la liste, le code soustrait le premier élément de la liste du total. Cela signifie que le résultat final sera toujours inférieur à la somme réelle des éléments de la liste par la valeur du premier élément.

### Correction

Pour corriger ce bug, il suffit de supprimer la ligne qui décrémente le total par le premier élément de la liste. Voici la version corrigée du code :

```python
def somme_liste(liste):
    total = 0
    for i in range(len(liste)):
        total += liste[i]
    return total

# Exemple d'utilisation
ma_liste = [1, 2, 3, 4]
print

### Résultat du Self-refine : Amélioration itérative

Le modèle a :
1. **Identifié le bug** : L'ajout de `+ 1` dans le `return` fausse le résultat
2. **Proposé une correction** : Retirer le `+ 1`
3. **Suggéré une amélioration** : Utiliser la fonction native `sum()` pour plus de simplicité

**Enseignement** : Le modèle possède une capacité de **méta-cognition** - il peut raisonner sur ses propres productions et les améliorer.

### Variante avancée : Self-refine avec role "developer"

Au lieu d'un processus en deux appels API, on peut utiliser le rôle `developer` pour configurer le comportement d'auto-amélioration directement.

### Bilan du Self-refine

Le processus Self-refine en deux étapes démontre la capacité du modèle à s'**auto-améliorer** :

**Première étape** : Génération intentionnellement bugguée
- Le modèle crée du code avec un bug volontaire (par exemple, division par zéro, mauvaise initialisation, etc.)

**Deuxième étape** : Critique et correction
- Le modèle analyse son propre code
- Identifie le bug
- Propose une version corrigée
- Explique la nature du problème

**Enseignements** :
1. **Méta-cognition** : Le LLM peut raisonner sur ses propres productions
2. **Amélioration itérative** : Chaque passe peut affiner la qualité
3. **Détection de bugs** : Utile pour du code review automatisé

**Applications pratiques** :
- Génération de code robuste (plusieurs passes de correction)
- Rédaction de documents (brouillon → révision → version finale)
- Traduction (première traduction → révision → amélioration)

**Limite** : Chaque itération consomme des tokens et du temps. À utiliser pour des tâches critiques nécessitant haute qualité.

In [9]:
# ============================
# Cellule X (NOUVELLE) : Self-refine avec developer role
# ============================

messages_sr = [
    {
        "role": "developer",
        "content": (
            "You are a self-improving coding assistant. Whenever you provide code, "
            "you will automatically search for potential bugs or improvements "
            "and refine your output."
        )
    },
    {
        "role": "user",
        "content": (
            "Écris une fonction Python qui calcule la factorielle d'un nombre entier. "
            "Ensuite, relis-toi et corrige d'éventuels bugs."
        )
    }
]

response_self_refine = client.chat.completions.create(
    model=MODEL_NAME,
    messages=messages_sr,
    max_tokens=300,
    temperature=0.3
)

print("=== Self-refine avec 'developer role' ===")
print(response_self_refine.choices[0].message.content)


=== Self-refine avec 'developer role' ===
Voici une fonction Python qui calcule la factorielle d'un nombre entier :

```python
def factorielle(n):
    if not isinstance(n, int) or n < 0:
        raise ValueError("Le nombre doit être un entier non négatif.")
    if n == 0 or n == 1:
        return 1
    resultat = 1
    for i in range(2, n + 1):
        resultat *= i
    return resultat
```

### Vérification et améliorations :

1. **Validation des entrées** : La fonction vérifie si l'entrée est un entier non négatif. Si ce n'est pas le cas, elle lève une exception `ValueError`.

2. **Cas de base** : La factorielle de 0 et 1 est bien gérée en retournant 1.

3. **Boucle de calcul** : La boucle commence à 2, ce qui est correct, car multiplier par 1 est redondant.

4. **Performance** : Pour les très grands nombres, cette approche pourrait être améliorée en utilisant des techniques avancées comme la récursion avec mémorisation ou des bibliothèques spécialisées, mais pour des usages courants,

### Interprétation du Self-refine avec role "developer"

Le rôle `developer` (introduit avec les modèles de raisonnement) permet de définir le comportement global du modèle, contrairement au rôle `system` qui était utilisé auparavant.

**Différence clé** :
- **Role "system"** (ancienne API) : Instructions générales, parfois ignorées par le modèle
- **Role "developer"** (nouvelle API) : Instructions prioritaires, mieux respectées

Dans cet exemple :
- Le modèle génère la fonction factorielle
- Il vérifie automatiquement les bugs potentiels
- Il propose des améliorations (gestion d'erreurs, performance, alternatives)

**Observation** : La fonction générée inclut directement :
- Vérification du type (`isinstance`)
- Gestion des nombres négatifs
- Version alternative avec `math.factorial`

C'est plus efficace que le Self-refine en deux passes, mais nécessite un modèle récent supportant le rôle `developer`.

In [10]:
# ============================
# Cellule 9 : Prompt interactif
# ============================

# En mode batch (BATCH_MODE=true dans .env), cette cellule utilise des exemples prédéfinis
# En mode interactif, elle permet de tester le modèle en boucle

if BATCH_MODE:
    # Mode batch: exécuter des exemples simples
    test_prompts = ["Bonjour!", "Quelle est la capitale de la France?"]
    for prompt in test_prompts:
        print(f"[BATCH] Prompt: {prompt}")
        resp = client.chat.completions.create(
            model=MODEL_NAME,
            messages=[{"role":"user","content":prompt}],
            max_tokens=200,
            temperature=0.7
        )
        print(f"Réponse: {resp.choices[0].message.content}\n")
    print("Mode batch terminé.")
else:
    while True:
        user_input = input("Tape ton prompt ('exit' pour quitter) : ")
        if user_input.strip().lower() in ["exit", "quit"]:
            print("Fin de l'interaction.")
            break

        resp = client.chat.completions.create(
            model=MODEL_NAME,
            messages=[{"role":"user","content":user_input}],
            max_tokens=200,
            temperature=0.7
        )

        print("\n=== Réponse du modèle ===")
        print(resp.choices[0].message.content)
        print("---------------------------------------------------\n")

[BATCH] Prompt: Bonjour!
Réponse: Bonjour! Comment puis-je vous aider aujourd'hui?

[BATCH] Prompt: Quelle est la capitale de la France?
Réponse: La capitale de la France est Paris.

Mode batch terminé.


### Interprétation des tests interactifs

Cette cellule démontre le **mode batch** pour tester le modèle sans interaction manuelle.

**Résultats observés** :
- **Prompt simple** : "Bonjour!" → Réponse polie et ouverture de dialogue
- **Question factuelle** : "Quelle est la capitale de la France?" → Réponse directe et précise

**Points clés** :
1. **Température 0.7** : Équilibre entre cohérence et créativité
2. **Max tokens 200** : Limite les réponses pour réduire les coûts
3. **Messages stateless** : Chaque prompt est indépendant, pas de mémoire entre appels

**Différence avec la cellule suivante** : La prochaine cellule introduit la **mémoire de conversation** (accumulation de l'historique), essentielle pour les chatbots.

In [11]:
# ============================
# Cellule 9 : Prompt interactif avec mémoire de chat
# ============================

# En mode batch, cette cellule utilise un dialogue prédéfini
# En mode interactif, elle permet un échange avec mémoire

if BATCH_MODE:
    # Mode batch: simuler une conversation avec mémoire
    batch_conversation = [
        "Je m'appelle Alice",
        "Comment je m'appelle?",
    ]
    current_messages = []

    for user_input in batch_conversation:
        print(f"[BATCH] User: {user_input}")
        resp = client.chat.completions.create(
            model=MODEL_NAME,
            messages=current_messages + [{"role":"user","content":user_input}],
            max_tokens=200,
            temperature=0.7,
        )
        assistant_message = resp.choices[0].message.content
        print(f"Assistant: {assistant_message}\n")
        current_messages.append({"role":"user","content":user_input})
        current_messages.append({"role":"assistant","content":assistant_message})

    print("Mode batch (mémoire) terminé.")
else:
    user_input = ""
    current_messages = []

    while True:
        user_input = input("Tape ton prompt ('exit' pour quitter) : ")
        if user_input.strip().lower() in ["exit", "quit"]:
            print("Fin de l'interaction.")
            break
        print("\n=== message de l'utilisateur ===")
        resp = client.chat.completions.create(
            model=MODEL_NAME,
            messages = current_messages + [{"role":"user","content":user_input}],
            max_tokens=200,
            temperature=0.7,
        )

        print("\n=== Réponse du modèle ===")
        assistant_message = resp.choices[0].message.content
        print(assistant_message)
        print("---------------------------------------------------\n")
        current_messages.append({"role":"user","content":user_input})
        current_messages.append({"role":"assistant","content":assistant_message})

[BATCH] User: Je m'appelle Alice
Assistant: Bonjour, Alice ! Comment puis-je vous aider aujourd'hui ?

[BATCH] User: Comment je m'appelle?
Assistant: Vous avez dit que vous vous appelez Alice.

Mode batch (mémoire) terminé.


### Interprétation de la conversation avec mémoire

Cette cellule démontre la **mémoire conversationnelle** - le modèle se souvient du contexte précédent.

**Résultats observés** :
1. **Tour 1** : "Je m'appelle Alice" → Le modèle enregistre cette information dans `current_messages`
2. **Tour 2** : "Comment je m'appelle?" → Le modèle répond "Vous vous appelez Alice" grâce à l'historique

**Mécanisme technique** :
```python
current_messages = [
    {"role": "user", "content": "Je m'appelle Alice"},
    {"role": "assistant", "content": "Bonjour, Alice..."},
    {"role": "user", "content": "Comment je m'appelle?"},
    # Le modèle peut répondre grâce à l'historique complet
]
```

**Applications pratiques** :
- Chatbots conversationnels
- Assistants personnels
- Support client automatisé

**Gestion de l'historique** :
- **Court terme** : Garder tout l'historique (comme ici)
- **Long terme** : Tronquer les messages anciens ou résumer périodiquement
- **Tokens** : Attention au coût - chaque tour envoie TOUT l'historique à l'API

# Prompting pour Modèles de Raisonnement (2025)

Les modèles de raisonnement (o4-mini, gpt-5-thinking) représentent une évolution majeure. Contrairement aux modèles de chat, ils prennent le temps de "réfléchir" avant de répondre.

## Différences clés avec les modèles chat

| Aspect | Modèles Chat (gpt-4o) | Modèles Raisonnement (o4-mini) |
|--------|----------------------|-------------------------------|
| Temps de réponse | Rapide | Plus lent (réflexion) |
| Prompts | Détaillés, structurés | **Simples et directs** |
| Chain-of-thought | Demandé explicitement | **Intégré nativement** |
| Messages | system, user, assistant | **developer**, user, assistant |
| Paramètre spécial | temperature | **reasoning_effort** |

## Règle d'or : Simplifier les prompts

**Pour les modèles de raisonnement, des prompts simples fonctionnent mieux !**

Les modèles raisonnants sont capables de :
- Comprendre l'intention sans instructions détaillées
- Gérer les ambiguïtés intelligemment
- Demander des clarifications si nécessaire

**Éviter** : "Analyse ce problème en détaillant chaque étape de ton raisonnement..."
**Préférer** : "Résous ce problème."

In [ ]:
# ============================
# Cellule : Comparaison Chat vs Reasoning
# ============================

import time

probleme_complexe = """
Un fermier veut traverser une rivière avec un loup, une chèvre et un chou.
Son bateau ne peut transporter que lui et un objet à la fois.
Si le loup est laissé seul avec la chèvre, il la mange.
Si la chèvre est laissée seule avec le chou, elle le mange.
Comment le fermier peut-il tout transporter de l'autre côté?
"""

# Test avec gpt-5-mini (chat model)
print("=== gpt-5-mini (Chat Model) ===")
start = time.time()
response_chat = client.chat.completions.create(
    model="gpt-5-mini",
    messages=[{"role": "user", "content": probleme_complexe}],
    max_tokens=500,
    temperature=0.2
)
print(f"Temps: {time.time() - start:.2f}s")
print(response_chat.choices[0].message.content[:400] + "...")

# Test avec o4-mini (reasoning model) - si disponible
print("\n=== o4-mini (Reasoning Model) ===")
try:
    start = time.time()
    response_reason = client.chat.completions.create(
        model="o4-mini",
        messages=[
            {"role": "developer", "content": "Formatting re-enabled"},
            {"role": "user", "content": probleme_complexe}
        ],
        reasoning_effort="medium"
    )
    print(f"Temps: {time.time() - start:.2f}s")
    print(response_reason.choices[0].message.content[:400] + "...")
except Exception as e:
    print(f"o4-mini non disponible: {type(e).__name__}")
    print("Les modèles de raisonnement nécessitent un accès spécifique.")

### Analyse des résultats : Chat vs Reasoning

**Observations attendues** :

1. **Temps de réponse** :
   - Chat (gpt-4o-mini) : ~2-5 secondes
   - Reasoning (o4-mini) : ~10-30 secondes (réflexion interne)

2. **Qualité de la solution** :
   - Chat : Solution généralement correcte, mais peut manquer des optimisations
   - Reasoning : Solution optimale, avec explication détaillée

3. **Robustesse** :
   - Chat : Peut parfois donner des solutions incorrectes sur des variantes complexes
   - Reasoning : Vérifie sa solution avant de répondre, taux d'erreur plus faible

### Quand utiliser les modèles de raisonnement ?

**Préférer Chat** (gpt-4o, gpt-4o-mini) :
- Applications temps réel (chatbots)
- Tâches simples et courantes
- Budget temps/coût limité

**Préférer Reasoning** (o4-mini, gpt-5-thinking) :
- Mathématiques avancées
- Problèmes de logique complexes
- Code critique nécessitant validation
- Analyse approfondie

**Note** : Les modèles de raisonnement sont plus coûteux en tokens et en temps, mais offrent une qualité supérieure pour les tâches complexes.